# Football Data from Transfermarkt
- Kaggle Data URL

    - https://www.kaggle.com/datasets/davidcariboo/player-scores

## 데이터 설명
- Transfermarkt 웹사이트에서 스크레이핑한 각종 축구 데이터 (매주 한번 파일 업데이트)
    - 주요 대회 시즌별 60,000개 이상의 경기
    - 모든 대회의 400개 이상의 클럽들
    - 30,000 이상의 선수들
    - 400,000개 이상의 선수 가치 기록
    - 1,200,000개 이상의 선수 출전 기록

- `csv` 파일 설명
    - `apperances` : 선수 출장 기록
    - `club_games` : 클럽별 경기 홈팀, 어웨이팀 정보
    - `clubs` : 리그별 속해 있는 클럽
    - `competitions` : 대회 정보
    - `game_events` : 경기별 이벤트 정보 (카드, 득점, 어시스트 등)
    - `game_lineups` : 경기별 선수의 선발, 교체 명단 등재 여부
    - `games` : 경기에 대한 정보 (시즌, 라운드, 홈팀, 어웨이팀, 순위)
    - `player_valuations` : Transfermarkt 웹사이트에서 매긴 선수의 가치
    - `players` : 선수에 대한 세부 정보

- 데이터베이스 스키마
    <img src="https://raw.githubusercontent.com/dcaribou/transfermarkt-datasets/master/resources/diagram.svg?sanitize=true" width="1700">

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime
from datetime import timedelta
import missingno as msno

In [2]:
# 데이터 경로 지정
# C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data
apps_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/appearances.csv"
club_games_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/club_games.csv"
clubs_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/clubs.csv"
competitions_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/competitions.csv"
game_events_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/game_events.csv"
game_lineups_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/game_lineups.csv"
games_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/games.csv"
player_valuations_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/player_valuations.csv"
players_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/players.csv"

# 데이터셋 불러오기
apps_df = pd.read_csv(apps_path)
club_games_df = pd.read_csv(club_games_path)
clubs_df = pd.read_csv(clubs_path)
competitions_df = pd.read_csv(competitions_path)
game_events_df = pd.read_csv(game_events_path)
game_lineups_df = pd.read_csv(game_lineups_path)
games_df = pd.read_csv(games_path)
player_valuations_df = pd.read_csv(player_valuations_path)
players_df = pd.read_csv(players_path)

In [3]:
apps_df.head(3)

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45


In [4]:
club_games_df.head(3)

,game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
0,2320450,1468,0,NaN,Holger Bachthaler,24,2,NaN,Armin Veh,Home,0
1,2320460,1,3,NaN,Jürgen Luginger,86,1,NaN,Robin Dutt,Home,1
2,2320472,2036,4,NaN,Frank Schmidt,72,5,NaN,Alexander Schmidt,Home,0


In [109]:
clubs_df.tail(3)

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
423,800,atalanta-bergamo,Atalanta Bergamasca Calcio S.p.a.,IT1,NaN,25,26.5,17,68.0,16,Gewiss Stadium,21747,+€73.82m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/atalanta-berga...
424,979,moreirense-fc,Moreirense Futebol Clube,PO1,NaN,29,25.5,17,58.6,3,Estádio C. J. de Almeida Freitas,6153,€-2.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/moreirense-fc/...
425,984,west-bromwich-albion,West Bromwich Albion,GB1,NaN,25,27.8,8,32.0,4,The Hawthorns,26850,€-1.35m,NaN,2020,../data/raw/transfermarkt-scraper/2020/clubs.j...,https://www.transfermarkt.co.uk/west-bromwich-...


In [6]:
competitions_df.head(3)

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url
0,CIT,italy-cup,italy-cup,domestic_cup,domestic_cup,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/italy-cup/star...
1,NLSC,johan-cruijff-schaal,johan-cruijff-schaal,domestic_super_cup,other,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/johan-cruijff-...
2,GRP,kypello-elladas,kypello-elladas,domestic_cup,domestic_cup,56,Greece,GR1,europa,https://www.transfermarkt.co.uk/kypello-ellada...


In [7]:
game_events_df.head(3)

,game_event_id,date,game_id,minute,type,club_id,player_id,description,player_in_id,player_assist_id
0,2f41da30c471492e7d4a984951671677,2012-08-05,2211607,77,Cards,610,4425,"1. Yellow card , Mass confrontation",NaN,NaN
1,a72f7186d132775f234d3e2f7bc0ed5b,2012-08-05,2211607,77,Cards,383,33210,"1. Yellow card , Mass confrontation",NaN,NaN
2,b2d721eaed4692a5c59a92323689ef18,2012-08-05,2211607,3,Goals,383,36500,", Header, 1. Tournament Goal Assist: , Corner,...",NaN,56416.0


In [8]:
game_lineups_df.head(3)

,game_lineups_id,game_id,club_id,type,number,player_id,player_name,team_captain,position
0,b2dbe01c3656b06c8e23e9de714e26bb,2317258,610,substitutes,5,1443,Christian Poulsen,0,Defensive Midfield
1,b50a3ec6d52fd1490aab42042ac4f738,2317258,610,starting_lineup,4,5017,Niklas Moisander,0,Centre-Back
2,7d890e6d0ff8af84b065839966a0ec81,2317258,1090,substitutes,11,9602,Maarten Martens,0,Left Winger


In [9]:
games_df.head(3)

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2321044,L1,2013,2. Matchday,2013-08-18,16,23,2,1,1.0,...,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league
1,2321060,L1,2013,3. Matchday,2013-08-25,23,24,0,2,18.0,...,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league
2,2321086,L1,2013,6. Matchday,2013-09-21,4,16,1,1,15.0,...,Max-Morlock-Stadion,50000.0,Knut Kircher,https://www.transfermarkt.co.uk/1-fc-nuremberg...,4-2-3-1,4-2-3-1,1.FC Nuremberg,Borussia Dortmund,1:1,domestic_league


In [10]:
player_valuations_df.head(3)

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,405973,2000-01-20,150000,3057,BE1
1,342216,2001-07-20,100000,1241,SC1
2,3132,2003-12-09,400000,126,TR1


In [11]:
players_df.head(3)

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0


In [110]:
players_df[["name", "position", "sub_position"]]

,name,position,sub_position
0,Miroslav Klose,Attack,Centre-Forward
1,Roman Weidenfeller,Goalkeeper,Goalkeeper
2,Dimitar Berbatov,Attack,Centre-Forward
3,Lúcio,Defender,Centre-Back
4,Tom Starke,Goalkeeper,Goalkeeper
...,...,...,...
30490,Ozan Demirbağ,Attack,Left Winger
30491,Yusuf Yılmaz,Goalkeeper,Goalkeeper
30492,Sid Ahmed Aissaoui,Midfield,Central Midfield
30493,Erdem Çalık,Midfield,Central Midfield


### `.info()` 확인

In [12]:
# df.info() 확인

dfs = [apps_df, club_games_df, clubs_df, competitions_df, game_events_df, game_lineups_df, 
       games_df, player_valuations_df, players_df]

for df in dfs:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540638 entries, 0 to 1540637
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   appearance_id           1540638 non-null  object
 1   game_id                 1540638 non-null  int64 
 2   player_id               1540638 non-null  int64 
 3   player_club_id          1540638 non-null  int64 
 4   player_current_club_id  1540638 non-null  int64 
 5   date                    1540638 non-null  object
 6   player_name             1540637 non-null  object
 7   competition_id          1540638 non-null  object
 8   yellow_cards            1540638 non-null  int64 
 9   red_cards               1540638 non-null  int64 
 10  goals                   1540638 non-null  int64 
 11  assists                 1540638 non-null  int64 
 12  minutes_played          1540638 non-null  int64 
dtypes: int64(9), object(4)
memory usage: 152.8+ MB
None
<class 'pandas.core.

In [18]:
apps_copy = apps_df.copy()

### 사용 데이터셋
- 목표 : 선수의 시즌별 스탯 확인
    - `apperances` : 선수 출장 기록
    - `competitions` : 대회 정보
    - `players` : 선수에 대한 세부 정보
    
- 해볼 수 있는 것
    - 선수 스탯 예측
    - 선수 몸값 예측

In [163]:
apps_df.describe(include="all")

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
count,1540638,1.540638e+06,1.540638e+06,1.540638e+06,1.540638e+06,1540638,1540637,1540638,1.540638e+06,1.540638e+06,1.540638e+06,1.540638e+06,1.540638e+06
unique,1540638,NaN,NaN,NaN,NaN,3416,23472,43,NaN,NaN,NaN,NaN,NaN
top,2231978_38004,NaN,NaN,NaN,NaN,2020-10-04,Danilo,IT1,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,1795,1062,127333,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.045293e+06,1.855729e+05,2.967665e+03,3.642055e+03,NaN,NaN,NaN,1.486358e-01,3.806864e-03,9.627894e-02,7.582248e-02,6.956878e+01
std,NaN,5.774826e+05,1.704357e+05,7.534551e+03,9.289958e+03,NaN,NaN,NaN,3.670711e-01,6.158226e-02,3.316466e-01,2.862993e-01,2.979700e+01
min,NaN,2.211607e+06,1.000000e+01,1.000000e+00,-1.000000e+00,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,NaN,2.545933e+06,5.528200e+04,2.810000e+02,3.360000e+02,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.100000e+01
50%,NaN,2.981997e+06,1.299910e+05,8.260000e+02,9.310000e+02,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+01
75%,NaN,3.497453e+06,2.700510e+05,2.441000e+03,2.687000e+03,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+01


In [168]:
competitions_df.describe(include="all")

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url
count,43,43,43,43,43,43.000000,36,36,43,43
unique,43,42,42,11,4,NaN,14,14,1,43
top,CIT,premier-liga,premier-liga,first_tier,domestic_league,NaN,England,GB1,europa,https://www.transfermarkt.co.uk/italy-cup/star...
freq,1,2,2,14,14,NaN,4,4,43,1
mean,NaN,NaN,NaN,NaN,NaN,97.093023,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,69.766896,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,39.500000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,122.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,157.000000,NaN,NaN,NaN,NaN


In [170]:
players_df.describe()

,player_id,last_season,current_club_id,height_in_cm,market_value_in_eur,highest_market_value_in_eur
count,3.049500e+04,30495.000000,30495.000000,28410.000000,2.935200e+04,2.935200e+04
mean,3.142658e+05,2018.790589,4416.199115,182.250827,1.574004e+06,3.635859e+06
std,2.531242e+05,3.651044,10149.194332,6.841391,6.154743e+06,9.463744e+06
min,1.000000e+01,2012.000000,3.000000,18.000000,1.000000e+04,1.000000e+04
25%,9.598000e+04,2016.000000,403.000000,178.000000,1.000000e+05,2.750000e+05
50%,2.590250e+05,2020.000000,1075.000000,182.000000,2.500000e+05,8.000000e+05
75%,4.704080e+05,2022.000000,3057.000000,187.000000,6.500000e+05,2.800000e+06
max,1.229924e+06,2023.000000,83678.000000,207.000000,1.800000e+08,2.000000e+08


In [169]:
players_df.describe(include="object")

,first_name,last_name,name,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name
count,28505,30495,30495,30495,27773,28294,29919,30450,30321,30495,28122,19041,15115,30495,30495,30495,30495
unique,6596,22453,29847,29810,184,8227,182,8931,13,5,3,100,2671,24940,30495,14,424
top,David,García,Paulinho,paulinho,France,London,Spain,1996-01-19,Centre-Back,Defender,right,2024-06-30 00:00:00,Wasserman,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,TR1,Kilmarnock Football Club
freq,203,61,12,12,2185,434,1813,20,5357,9703,19733,5158,435,5556,1,3005,181


#### 사용 데이터셋 병합 함수 생성

In [174]:
# appearance_df, competitions_df, players_df merge 후 19-20 ~ 23-24 시즌 유럽 6대리그 데이터만 남김 (5시즌)
def app_comp_player_merge(apps_copy, competitions_df, players_df):
    # 데이터프레임의 date 컬럼 타입 datetime으로 변경
    apps_copy["date"] = pd.to_datetime(apps_copy["date"])
    
    # apps_copy와 competition_df competition_id를 기준으로 left join
    app_comp_df = pd.merge(apps_copy, competitions_df, how="left", on="competition_id")
    
    # 19-20 ~ 23-24 시즌 데이터만 남김 (5시즌)
    app_comp_df = app_comp_df[app_comp_df["date"]>="2019-06"]

    app_comp_player_df = pd.merge(app_comp_df, players_df, how="left", on="player_id")

    cols = ['game_id', 'player_id', 'player_club_id',
            'player_current_club_id', 'date', 'player_name', 'competition_id',
            'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
            'competition_code', 'name_x', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
            'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
            'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']    

    # 유럽 6대리그 (잉글랜드, 스페인, 이탈리아, 독일, 프랑스, 포르투갈 1부리그)만 필터링
    league_cond = ((app_comp_player_df.domestic_league_code=="GB1") | (app_comp_player_df.domestic_league_code=="ES1") | 
                   (app_comp_player_df.domestic_league_code=="IT1") | (app_comp_player_df.domestic_league_code=="L1") |
                   (app_comp_player_df.domestic_league_code=="FR1") | (app_comp_player_df.domestic_league_code=="PO1"))
    app_comp_player_df = app_comp_player_df[cols][league_cond].reset_index(drop=True)
    # app_comp_player_df = app_comp_player_df[league_cond].reset_index(drop=True)
    # a_cols = ["player_name", "last_season", "yellow_cards", "red_cards","goals", "assists",	"minutes_played"]
    
    # app_comp_player_df_a = app_comp_player_df[a_cols]
    # app_comp_player_df_b = app_comp_player_df.loc[:, ~app_comp_player_df.columns.isin(a_cols)]

    return app_comp_player_df

In [182]:
df_merged = app_comp_player_merge(apps_copy, competitions_df, players_df)
df_merged.head(2)

,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,...,last_season,country_of_citizenship,date_of_birth,sub_position,foot,height_in_cm,contract_expiration_date,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,3213705,243779,10140,2995,2019-07-27,Adriano Castanheira,POCP,0,0,0,...,2021,Portugal,1993-04-07,Left Winger,left,178.0,2023-06-30 00:00:00,FC Paços de Ferreira,200000.0,600000.0
1,3182926,100986,583,449,2019-08-03,Thomas Meunier,FRCH,1,0,0,...,2023,Belgium,1991-09-12,Right-Back,right,191.0,2025-06-30 00:00:00,Trabzonspor Kulübü,2500000.0,35000000.0


In [183]:
df_merged.to_csv("data/df_merged.csv", index=False)


In [176]:
df[df.player_name=="Aaron Cresswell"][["player_name", "date", "last_season"]]

,player_name,date,last_season
480,Aaron Cresswell,2019-08-10,2023
9028,Aaron Cresswell,2019-09-22,2023
11082,Aaron Cresswell,2019-09-28,2023
12485,Aaron Cresswell,2019-10-05,2023
15504,Aaron Cresswell,2019-10-26,2023
...,...,...,...
286266,Aaron Cresswell,2023-10-29,2023
290497,Aaron Cresswell,2023-11-12,2023
295965,Aaron Cresswell,2023-12-10,2023
304340,Aaron Cresswell,2024-01-16,2023


In [177]:
df[["player_name", "last_season", "yellow_cards", "red_cards","goals", "assists",	"minutes_played"]]

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played
0,Adriano Castanheira,2021,0,0,0,0,90
1,Thomas Meunier,2023,1,0,0,0,80
2,Marco Verratti,2022,0,0,0,0,90
3,Alphonse Areola,2023,0,0,0,0,90
4,Juan Bernat,2023,1,0,0,0,90
...,...,...,...,...,...,...,...
315272,Jhon Solís,2023,0,0,0,0,1
315273,Sávio,2023,1,0,2,0,90
315274,Aridane Hernández,2023,0,0,0,0,90
315275,Iván Balliu,2023,0,0,0,0,90


In [178]:
df[["player_name", "last_season", "yellow_cards", "red_cards","goals", "assists",	"minutes_played"]].groupby(["player_name", "last_season"]).sum().head(30)

,,yellow_cards,red_cards,goals,assists,minutes_played
player_name,last_season,,,,,
Aaron Boupendza,2020,0,0,0,0,90
Aaron Connolly,2022,0,0,5,3,2369
Aaron Cresswell,2023,17,0,5,13,11389
Aaron Donnelly,2023,0,0,0,0,90
Aaron Hickey,2023,22,1,5,2,6363
Aaron Lennon,2021,6,0,2,1,2305
Aaron Leya Iseka,2023,6,0,6,0,1776
Aaron Mooy,2022,3,0,2,2,2089
Aaron Ramsdale,2023,3,0,0,1,14400


In [179]:
df.columns

Index(['game_id', 'player_id', 'player_club_id', 'player_current_club_id',
       'date', 'player_name', 'competition_id', 'yellow_cards', 'red_cards',
       'goals', 'assists', 'minutes_played', 'competition_code', 'name_x',
       'country_id', 'country_name', 'domestic_league_code', 'last_season',
       'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot',
       'height_in_cm', 'contract_expiration_date', 'current_club_name',
       'market_value_in_eur', 'highest_market_value_in_eur'],
      dtype='object')

In [180]:
df.describe()

,game_id,player_id,player_club_id,player_current_club_id,date,yellow_cards,red_cards,goals,assists,minutes_played,country_id,last_season,height_in_cm,market_value_in_eur,highest_market_value_in_eur
count,3.152770e+05,3.152770e+05,315277.000000,315277.000000,315277,315277.000000,315277.000000,315277.000000,315277.000000,315277.000000,315277.000000,315277.000000,314896.000000,3.152050e+05,3.152050e+05
mean,3.625008e+06,2.857655e+05,1503.158762,1633.820932,2021-12-04 10:41:49.039353088,0.143284,0.003771,0.092376,0.071702,66.247833,110.754029,2022.517865,182.396712,1.091674e+07,2.020709e+07
min,3.182926e+06,5.320000e+02,3.000000,3.000000,2019-07-27 00:00:00,0.000000,0.000000,0.000000,0.000000,1.000000,40.000000,2019.000000,162.000000,1.000000e+04,2.500000e+04
25%,3.413037e+06,1.307650e+05,237.000000,237.000000,2020-11-28 00:00:00,0.000000,0.000000,0.000000,0.000000,45.000000,50.000000,2022.000000,178.000000,1.200000e+06,4.000000e+06
50%,3.592194e+06,2.577320e+05,631.000000,621.000000,2021-12-11 00:00:00,0.000000,0.000000,0.000000,0.000000,86.000000,136.000000,2023.000000,183.000000,4.000000e+06,1.200000e+07
75%,3.844924e+06,3.952370e+05,1160.000000,1158.000000,2023-02-01 00:00:00,0.000000,0.000000,0.000000,0.000000,90.000000,157.000000,2023.000000,187.000000,1.300000e+07,2.600000e+07
max,4.279372e+06,1.178474e+06,53286.000000,83678.000000,2024-02-26 00:00:00,2.000000,1.000000,4.000000,4.000000,120.000000,189.000000,2023.000000,206.000000,1.800000e+08,2.000000e+08
std,2.997178e+05,1.889927e+05,3020.766715,3778.138365,NaN,0.359195,0.061295,0.322131,0.278555,30.915701,56.041571,0.906190,6.590956,1.786123e+07,2.411517e+07


In [181]:
df["last_season"].value_counts()

last_season
2023    225265
2022     50681
2021     22101
2020     11797
2019      5433
Name: count, dtype: int64

In [141]:
df[(df.last_season==2023) & (df.player_name=="Bukayo Saka")]

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url_y,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
9046,3219074_433177,3219074,433177,11,11,2019-09-22,Bukayo Saka,GB1,0,0,...,left,178.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/bukayo-saka/pr...,GB1,Arsenal Football Club,120000000.0,120000000.0
11852,3219068_433177,3219068,433177,11,11,2019-09-30,Bukayo Saka,GB1,0,0,...,left,178.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/bukayo-saka/pr...,GB1,Arsenal Football Club,120000000.0,120000000.0
13133,3219400_433177,3219400,433177,11,11,2019-10-06,Bukayo Saka,GB1,0,0,...,left,178.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/bukayo-saka/pr...,GB1,Arsenal Football Club,120000000.0,120000000.0
14716,3219386_433177,3219386,433177,11,11,2019-10-21,Bukayo Saka,GB1,1,0,...,left,178.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/bukayo-saka/pr...,GB1,Arsenal Football Club,120000000.0,120000000.0
16099,3219385_433177,3219385,433177,11,11,2019-10-27,Bukayo Saka,GB1,0,0,...,left,178.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/bukayo-saka/pr...,GB1,Arsenal Football Club,120000000.0,120000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307732,4095289_433177,4095289,433177,11,11,2024-01-30,Bukayo Saka,GB1,0,0,...,left,178.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/bukayo-saka/pr...,GB1,Arsenal Football Club,120000000.0,120000000.0
309128,4095297_433177,4095297,433177,11,11,2024-02-04,Bukayo Saka,GB1,0,0,...,left,178.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/bukayo-saka/pr...,GB1,Arsenal Football Club,120000000.0,120000000.0
311113,4095318_433177,4095318,433177,11,11,2024-02-11,Bukayo Saka,GB1,0,0,...,left,178.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/bukayo-saka/pr...,GB1,Arsenal Football Club,120000000.0,120000000.0
312039,4095321_433177,4095321,433177,11,11,2024-02-17,Bukayo Saka,GB1,1,0,...,left,178.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/bukayo-saka/pr...,GB1,Arsenal Football Club,120000000.0,120000000.0


In [ ]:
# # appearance_df, competitions_df merge 후 23-24시즌 유럽 6대리그 데이터만 남김
# def app_comp_merge_2324(apps_copy, competitions_df):
#     # 데이터프레임의 date 컬럼 타입 datetime으로 변경
#     apps_copy["date"] = pd.to_datetime(apps_copy["date"])
    
#     # apps_copy와 competition_df competition_id를 기준으로 left join
#     app_comp_df = pd.merge(apps_copy, competitions_df, how="left", on="competition_id")
    
#     # 19-20 ~ 23-24 시즌 데이터만 남김 (4시즌)
#     app_comp_df = app_comp_df[app_comp_df["date"]>="2019-06"]
    
#     # 유럽 6대리그 (잉글랜드, 스페인, 이탈리아, 독일, 프랑스, 포르투갈 1부리그)만 필터링
#     league_cond = ((app_comp_df.domestic_league_code=="GB1") | (app_comp_df.domestic_league_code=="ES1") | 
#                    (app_comp_df.domestic_league_code=="IT1") | (app_comp_df.domestic_league_code=="L1") |
#                    (app_comp_df.domestic_league_code=="FR1") | (app_comp_df.domestic_league_code=="PO1"))
#     app_comp_df =app_comp_df[league_cond]
#     return app_comp_df

In [98]:
players_df

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,...,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lucio/profil/s...,IT1,Juventus Football Club,200000.0,24500000.0
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,...,right,194.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/tom-starke/pro...,L1,FC Bayern München,100000.0,3000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30490,1201294,Ozan,Demirbağ,Ozan Demirbağ,2023,3840,ozan-demirbag,Türkiye,Tokat,Türkiye,...,right,178.0,2026-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ozan-demirbag/...,TR1,Adana Demirspor Kulübü,50000.0,50000.0
30491,1201580,Yusuf,Yılmaz,Yusuf Yılmaz,2023,6890,yusuf-yilmaz,Türkiye,Istanbul,Türkiye,...,right,193.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/yusuf-yilmaz/p...,TR1,İstanbul Başakşehir Futbol Kulübü,NaN,NaN
30492,1214946,Sid Ahmed,Aissaoui,Sid Ahmed Aissaoui,2023,2410,sid-ahmed-aissaoui,Algeria,Blida,Algeria,...,right,183.0,2026-06-30 00:00:00,Aniss Benchabane,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/sid-ahmed-aiss...,RU1,PFK CSKA Moskva,150000.0,150000.0
30493,1225269,Erdem,Çalık,Erdem Çalık,2023,3209,erdem-calik,Türkiye,Istanbul,Türkiye,...,right,NaN,2026-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/erdem-calik/pr...,TR1,Pendikspor,50000.0,50000.0
